In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from time import time

from numpy import (
    dot, float32 as REAL
)
import numpy as np

Cargar Modelos originales

In [2]:
modeloD2V = Doc2Vec.load("./Modelos/modelo2_FINALD2V.model")
modeloW2V = Word2Vec.load("./Modelos/modelo10FINAL.model")

# Vectores a ElasticSearch

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch()
es.ping()

True

**creación de indices Palabras**

In [9]:
mapping_palabras="""
{
    "mappings":{
        "properties":{
            "index":{"type": "integer"},
            "palabra":{"type": "keyword"},
            "vector":{
                "type": "dense_vector",
                "dims": 300
            },
            "norm_vec":{
                "type": "dense_vector",
                "dims": 300
            }
        }
    }
}
"""
# es.indices.delete(index='palabras')
es.indices.create(body=mapping_palabras, index='palabras')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'palabras'}

**creación de indices Documentos**

In [67]:
mapping_docs="""
{
    "mappings":{
        "properties":{
            "tag":{"type": "integer"},
            "vector":{
                "type": "dense_vector",
                "dims": 300
            },
            "norm_vec":{
                "type": "dense_vector",
                "dims": 300
            }
        }
    }
}
"""
# es.indices.delete(index='documentos')
es.indices.create(body=mapping_docs, index='documentos')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'documentos'}

In [13]:
print(es.indices.get('palabras'), end='\n\n')
print(es.indices.get('documentos'))

{'palabras': {'aliases': {}, 'mappings': {'properties': {'index': {'type': 'integer'}, 'norm_vec': {'type': 'dense_vector', 'dims': 300}, 'palabra': {'type': 'keyword'}, 'vector': {'type': 'dense_vector', 'dims': 300}}}, 'settings': {'index': {'creation_date': '1608162403100', 'number_of_shards': '1', 'number_of_replicas': '1', 'uuid': 'v2mqowdGSuGXzRYCOihh3A', 'version': {'created': '7060099'}, 'provided_name': 'palabras'}}}}

{'documentos': {'aliases': {}, 'mappings': {'properties': {'norm_vec': {'type': 'dense_vector', 'dims': 300}, 'tag': {'type': 'integer'}, 'vector': {'type': 'dense_vector', 'dims': 300}}}, 'settings': {'index': {'creation_date': '1607123586198', 'number_of_shards': '1', 'number_of_replicas': '1', 'uuid': 'hFkHM_O3TWmms3zkhR36BQ', 'version': {'created': '7060099'}, 'provided_name': 'documentos'}}}}


**Datos para Palabras**

In [25]:
type(modeloW2V.wv.vocab['diagnostico'].index)

int

In [28]:
# for i, (vector, vector_norm, palabra) in enumerate(zip(modeloW2V.wv.vectors, modeloW2V.wv.vectors_norm, modeloW2V.wv.vocab.keys())):

#     data = {
#         "index":i,
#         "palabra":palabra,
#         "vector":vector,
#         "norm_vec":vector_norm
#     }

#     es.index(index="palabras", body=data, id=i)

# print(es.count(index='palabras'))

for key, value in modeloW2V.wv.vocab.items():
    
    index = value.index
    vector = modeloW2V.wv.vectors[index]
    vector_norm = modeloW2V.wv.vectors_norm[index]
    data = {
        "doc":{
            "index":index,
            "palabra":key,
            "vector":vector,
            "norm_vec":vector_norm
        }
    }
    
    es.index(index="palabras", body=data, id=index)
    es.update(index="palabras", body=data, id=index)
    
    
print(es.count(index='palabras'))


{'count': 11445, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


**Datos Documentos D2V**

In [44]:
for i, (vector, vector_norm) in enumerate(zip(modeloD2V.docvecs.vectors_docs, modeloD2V.docvecs.vectors_docs_norm)):

    data = {
        "tag":i+1,
        "vector":vector,
        "norm_vec":vector_norm
    }

    es.index(index="documentos", body=data, id=i+1)

print(es.count(index='documentos'))

{'count': 451, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


# Calculando algoritmos desde Elastic

In [10]:
palabra = es.get(index='palabras', id=1)['_source']
type(palabra['index'])

int

### W2V Most Similar

Consultamos los vectores correspondientes a las palabras a buscar

In [68]:
query_vecs = {
    "query":{
        "bool":{
            "should":[
                {"term": { "palabra": "ontologias" }},
                {"term": { "palabra": "randompalabra" }}
            ]
        }
    },
    "_source": ["palabra", "norm_vec"]
}
vectores = es.search(index='palabras', body=query_vecs)['hits']['hits']

Calculamos la Media de los vectores y transformamos a vector unitario

In [48]:
norm_vecs = [vector['_source']['norm_vec'] for vector in vectores]
vector_mean = np.array(norm_vecs).mean(axis=0)
unit_vec = vector_mean / np.linalg.norm(vector_mean)

Calculamos producto Punto en ElasticSearch y miramos el puntaje

In [49]:
query_sims = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec
        }
      }
    }
  },
  "size": 11,
  "_source": ["palabra", "index"]
}
most_similar = es.search(index='palabras', body=query_sims)['hits']['hits']
most_similar = [(palabra['_source']['index'],palabra['_source']['palabra'], palabra['_score']) for palabra in most_similar]
most_similar

[(2202, 'ontologias', 1.0),
 (4918, 'gestor', 0.85891134),
 (6645, 'sparql', 0.8413498),
 (6680, 'umayux', 0.8379714),
 (11329, 'buscador', 0.8294513),
 (4933, 'debilmente', 0.82685596),
 (11195, 'solicitada', 0.82475597),
 (6646, 'rdf', 0.8210847),
 (11316, 'evacuacion', 0.81534725),
 (11379, 'maskana', 0.8088352),
 (6643, 'motor', 0.80510694)]

In [70]:
words = ['gestor', 'conocimiento']
similar_words = modeloW2V.wv.most_similar(words, topn=10)
similar_words

[('debilmente', 0.77090984582901),
 ('umayux', 0.7446715831756592),
 ('sgbd', 0.7438246011734009),
 ('sparql', 0.7421228885650635),
 ('rdf', 0.7153308391571045),
 ('postgresql', 0.7092360854148865),
 ('ontologias', 0.7078660726547241),
 ('fundamentandose', 0.699651837348938),
 ('solicitada', 0.6902329921722412),
 ('maskana', 0.6835665702819824)]

### D2V Most Similar

Inferimos el Vector y calculamos la Media, luego se transforma  vector unitario

In [77]:
infer_vec = [modeloD2V.infer_vector(['gestor','conocimiento'])]
vec_mean_doc = np.array(infer_vec).mean(axis=0)
unit_vec_doc = vec_mean_doc / np.linalg.norm(vec_mean_doc)

Realizamos la consulta a Elastic con producto Punto

In [78]:
query_sims_docs = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec_doc
        }
      }
    }
  },
  "size": 10,
  "_source": ["tag"]
}

most_similar_docs = es.search(index='documentos', body=query_sims_docs)['hits']['hits']
most_similar_docs = [(doc['_source']['tag'], doc['_score']) for doc in most_similar_docs]
# most_similar_docs = [doc['_source']['tag'] for doc in most_similar]
most_similar_docs

[(443, 0.6676931),
 (422, 0.62167925),
 (438, 0.6189302),
 (417, 0.6073199),
 (435, 0.6052244),
 (139, 0.55522835),
 (337, 0.55080765),
 (65, 0.5423171),
 (445, 0.5416863),
 (21, 0.54074764)]

In [76]:
infer = modeloD2V.infer_vector(['gestor','conocimiento'])
sims_docs = modeloD2V.docvecs.most_similar([infer], topn=10)
for doc in sims_docs:
    print(f'({doc[0]+1}, {doc[1]})')

(443, 0.6732131242752075)
(438, 0.6272218227386475)
(417, 0.6206049919128418)
(422, 0.6184952855110168)
(435, 0.6097984910011292)
(139, 0.5551089644432068)
(337, 0.5512728691101074)
(425, 0.5449885129928589)
(445, 0.5434989929199219)
(444, 0.5386724472045898)
